<a href="https://colab.research.google.com/github/brianmulyadi/Designing-Data-Products/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import requests

In [2]:
# connect to ProPublica API
headers={'X-API-Key':'Guj6tsYRzPYaWECtImyu0NVttV3CrKbmD4NfXqHv'}
url = 'https://api.propublica.org/congress/v1/117/bills/hr1.json'
response = requests.get(url, headers=headers)
print(response.status_code)


200


In [31]:
# get congress members data
member_url = 'https://api.propublica.org/congress/v1/117/house/members.json'
response = requests.get(member_url, headers=headers)
json = response.json()

# generate list and enter into dataframe
memberId = []
fullName= []
dateOfBirth = []
gender = []
party = []
leadershipRole = []
dwNominate = []
seniority = []
votesAgainstParty = []

foo = 1

for x in json['results'][0]['members']:
  memberId.append(x['id'])
  fullName.append(x['first_name'] + " " + x['last_name'])
  dateOfBirth.append(x['date_of_birth'])
  gender.append(x['gender'])
  party.append(x['party'])
  leadershipRole.append(x['leadership_role'])
  dwNominate.append(x['dw_nominate'])
  seniority.append(x['seniority'])
  votesAgainstParty.append(x.get('votes_against_party_pct'))


member_list = list(zip(memberId, fullName, dateOfBirth, gender, party, leadershipRole, dwNominate, seniority, votesAgainstParty))
memberDf = pd.DataFrame(member_list, columns=['memberId', 'fullName', 'dateOfBirth', 'gender', 'party', 'leadershipRole', 'dwNominate', 'seniority', 'votesAgainstParty'])
memberDf.head()

,memberId,fullName,dateOfBirth,gender,party,leadershipRole,dwNominate,seniority,votesAgainstParty
0,A000370,Alma Adams,1946-05-27,F,D,None,-0.465,10,0.87
1,A000055,Robert Aderholt,1965-07-22,M,R,None,0.380,26,3.65
2,A000371,Pete Aguilar,1979-06-19,M,D,House Democratic Caucus Vice Chair,-0.296,8,1.09
3,A000372,Rick Allen,1951-11-07,M,R,None,0.699,8,8.97
4,A000376,Colin Allred,1983-04-15,M,D,None,NaN,4,1.75


In [36]:
memberDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   memberId           454 non-null    object 
 1   fullName           454 non-null    object 
 2   dateOfBirth        454 non-null    object 
 3   gender             454 non-null    object 
 4   party              454 non-null    object 
 5   leadershipRole     74 non-null     object 
 6   dwNominate         295 non-null    float64
 7   seniority          454 non-null    object 
 8   votesAgainstParty  453 non-null    float64
dtypes: float64(2), object(7)
memory usage: 32.0+ KB


In [38]:
from google.colab import files
memberDf.to_csv('members.csv')
files.download('members.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
# generate list and enter into dataframe
billId = []
billTitle= []
policyArea = []
introduceDate = []
daysProcessed = []
sponsorParty = []
sponsorGender = []
sponsorBill = []
sponsorAgainst = []
cosponsors = []
cosponsorsDem = []
cosponsorsRep = []


for x in range(1, 1000):
  url = 'https://api.propublica.org/congress/v1/117/bills/hr' + str(x) + '.json'
  response = requests.get(url, headers=headers)
  json = response.json()
  billId.append(json['results'][0]['bill_slug'])
  billTitle.append(json['results'][0]['short_title'])
  policyArea.append(json['results'][0]['primary_subject'])
  introduceDate.append(json['results'][0]['introduced_date'])
  daysProcessed.append(json['results'][0]['house_passage'])
  sponsorParty.append(json['results'][0]['sponsor_party'])
  sponsorGender.append(json['results'][0]['sponsor_id'])
  sponsorBill.append(json['results'][0]['sponsor_id'])
  sponsorAgainst.append(json['results'][0]['sponsor_id'])
  cosponsors.append(json['results'][0]['cosponsors'])
  cosponsorsDem.append(json['results'][0]['cosponsors_by_party'].get('D'))
  cosponsorsRep.append(json['results'][0]['cosponsors_by_party'].get('R'))


main_list = list(zip(billId, billTitle, policyArea, introduceDate, daysProcessed, sponsorParty, sponsorGender, sponsorBill, sponsorAgainst, cosponsors, cosponsorsDem, cosponsorsRep))

df = pd.DataFrame(main_list, columns=['billId', 'billTitle', 'policyArea', 'introduceDate', 'daysProcessed', 'sponsorParty', 'sponsorGender', 'sponsorBill', 'sponsorAgainst', 'cosponsors', 'cosponsorsDem', 'cosponsorsRep'])

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   billId          999 non-null    object 
 1   billTitle       999 non-null    object 
 2   policyArea      999 non-null    object 
 3   introduceDate   999 non-null    object 
 4   daysProcessed   77 non-null     object 
 5   sponsorParty    999 non-null    object 
 6   sponsorGender   999 non-null    object 
 7   sponsorBill     999 non-null    object 
 8   sponsorAgainst  999 non-null    object 
 9   cosponsors      999 non-null    int64  
 10  cosponsorsDem   532 non-null    float64
 11  cosponsorsRep   477 non-null    float64
dtypes: float64(2), int64(1), object(9)
memory usage: 93.8+ KB


In [34]:
df.tail()

,billId,billTitle,policyArea,introduceDate,daysProcessed,sponsorParty,sponsorGender,sponsorBill,sponsorAgainst,cosponsors,cosponsorsDem,cosponsorsRep
994,hr995,To amend the Child Abuse Prevention and Treatm...,Families,2021-02-11,None,D,H001090,H001090,H001090,2,NaN,2.0
995,hr996,Congress Commission Act,Congress,2021-02-11,None,D,H000324,H000324,H000324,4,4.0,NaN
996,hr997,English Language Unity Act of 2021,Government Operations and Politics,2021-02-11,None,R,G000552,G000552,G000552,28,NaN,28.0
997,hr998,Offshore Wind Jobs and Opportunity Act,Labor and Employment,2021-02-11,None,D,K000375,K000375,K000375,27,27.0,NaN
998,hr999,Wild Olympics Wilderness and Wild and Scenic R...,Public Lands and Natural Resources,2021-02-11,None,D,K000381,K000381,K000381,0,NaN,NaN


In [35]:
from google.colab import files
df.to_csv('1-999.csv')
files.download('1-999.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>